In [ ]:
!gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo

Downloading...
From: https://drive.google.com/uc?id=1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo
To: /content/SemEval2025_task11_processed.zip
100% 2.78M/2.78M [00:00<00:00, 89.9MB/s]


In [ ]:
!rm -rf processed_data
!unzip -q SemEval2025_task11_processed.zip

In [ ]:
!pip install -q transformers[torch] accelerate -U
!pip install -q datasets
!pip install -q transformers
!pip install -q iterative-stratification

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import os
from os import path
from tqdm.notebook import tqdm
from tqdm import tqdm
import string
import copy
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy
from datasets import Dataset,load_dataset, DatasetDict
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import classification_report,accuracy_score, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from scipy.special import expit


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
models_list_1=[
             "funnel-transformer/medium",
             "google/electra-base-discriminator",
             "facebook/bart-base",
        ]

models_name_1=[
"funnel-transformer-medium",
'electra-base',
"bart-base",
]

models_list_2=[
             "facebook/bart-large",
              "FacebookAI/xlm-roberta-base",
          ]

models_name_2=[
"bart-large",
"XLM-Roberta-base",
]

dels_list_3=[
      "FacebookAI/xlm-roberta-large",
     "google-bert/bert-base-multilingual-uncased",
          ]

models_name_3=[
"XLM-Roberta-large",
"bert",
]

models_list_4=[

     "distilbert/distilroberta-base",
     "distilbert/distilbert-base-uncased",
     "distilbert/distilbert-base-multilingual-cased",
      ]


models_name_4=[
"distil-roberta-base",
"distil-bert",
"distil-bert multilingual",
]

models_list_5=[
     "microsoft/mdeberta-v3-base",
     "albert/albert-base-v2",
     "xlnet/xlnet-base-cased",
     "google-t5/t5-base",
      ]
models_name_5=[
"DeBERTAV3",
"albert-base",
"xlnet-base",
"t5",
]

In [ ]:
kind="train"
task="track_a"
langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]
# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"

# train_data=pd.read_csv(processed_path)


In [ ]:
all_csv=pd.DataFrame()
for lang in langs:
    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    if not os.path.isfile(processed_path):
      print(processed_path)
      continue
    train_data=pd.read_csv(processed_path)
    train_data.columns = train_data.columns.str.lower()
    all_csv = pd.concat([all_csv, train_data],ignore_index=True)



processed_data/train/track_a/sum.csv


In [ ]:
train_data=all_csv.drop(columns={"unnamed: 0"})
train_data=train_data.fillna(0)
train_data

,anger,disgust,fear,joy,sadness,clean_message,surprise
0,0,0.0,0,1,0,ons sal gesondheidsprogramme in skole herinstel,0.0
1,0,0.0,0,1,0,ons moet verstaan dat ons in landbou een groot...,0.0
2,0,0.0,0,1,0,hulle is die trots van die boland en hulle hoo...,0.0
3,0,0.0,0,0,0,ek sou nog baie voorbeelde vir u kon noem,0.0
4,0,0.0,0,1,0,in ons bedryf loon dit gewoonlik om geduldig t...,0.0
...,...,...,...,...,...,...,...
25557,0,0.0,0,0,1,ሕማቅ ስማዕ ጽንዓት ይሃብ ቤተስብን ምቅርብ መቃልስቱን መንግስተ ሰማይ የ...,0.0
25558,1,0.0,0,0,0,ኣጃንዳ ክህቡና ደልዮም ዮም ዞም ሌያቡ,0.0
25559,0,0.0,0,0,0,ኣብ መንጎ ኤርትራን ኢትዮጵያን ተፈጥሩ ዘሎ ሰላም ዝንቅንቆ የለን erit...,0.0
25560,0,0.0,0,0,0,ብሪጣንያ ካብ ፈረንሳይ ዝሰግሩ ስደተኛታት ንምሓዝ ድሮንን ራዳርን ክጥቀም...,0.0


In [ ]:
train_data = train_data[
    train_data['clean_message'].isnull()==False]
train_data.reset_index()

,index,anger,disgust,fear,joy,sadness,clean_message,surprise
0,0,0,0.0,0,1,0,ons sal gesondheidsprogramme in skole herinstel,0.0
1,1,0,0.0,0,1,0,ons moet verstaan dat ons in landbou een groot...,0.0
2,2,0,0.0,0,1,0,hulle is die trots van die boland en hulle hoo...,0.0
3,3,0,0.0,0,0,0,ek sou nog baie voorbeelde vir u kon noem,0.0
4,4,0,0.0,0,1,0,in ons bedryf loon dit gewoonlik om geduldig t...,0.0
...,...,...,...,...,...,...,...,...
25557,25557,0,0.0,0,0,1,ሕማቅ ስማዕ ጽንዓት ይሃብ ቤተስብን ምቅርብ መቃልስቱን መንግስተ ሰማይ የ...,0.0
25558,25558,1,0.0,0,0,0,ኣጃንዳ ክህቡና ደልዮም ዮም ዞም ሌያቡ,0.0
25559,25559,0,0.0,0,0,0,ኣብ መንጎ ኤርትራን ኢትዮጵያን ተፈጥሩ ዘሎ ሰላም ዝንቅንቆ የለን erit...,0.0
25560,25560,0,0.0,0,0,0,ብሪጣንያ ካብ ፈረንሳይ ዝሰግሩ ስደተኛታት ንምሓዝ ድሮንን ራዳርን ክጥቀም...,0.0


In [ ]:
#update label column name
label_columns =['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']


y = train_data[label_columns].values

mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=1)
train_indices, valid_indices = next(mskf.split(train_data, y))

train = train_data.iloc[train_indices].reset_index(drop=True)
valid = train_data.iloc[valid_indices].reset_index(drop=True)

x_train, y_train = train['clean_message'].values.tolist(), train[label_columns].values.tolist()
x_valid, y_valid = valid['clean_message'].values.tolist(), valid[label_columns].values.tolist()

y_train_df = pd.DataFrame(y_train, columns=label_columns)

label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)
label_distribution.columns = ['count_0', 'count_1']
print(label_distribution)

          count_0  count_1
Anger        1948      266
Fear          926     1288
Joy          1675      539
Sadness      1512      702
Surprise     1542      672


In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH):
  train_encodings = tokenizer(x_train, truncation=True, padding="max_length", max_length=MAX_LENGTH)
  valid_encodings = tokenizer(x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH)
  train_dataset = Dataset(train_encodings, y_train)
  valid_dataset = Dataset(valid_encodings, y_valid)
  return train_dataset,valid_dataset

def train_arg():
    training_args = TrainingArguments(
    output_dir="./output/best-model-v1",
    report_to=None,
    num_train_epochs=7,
    gradient_accumulation_steps=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy=IntervalStrategy.STEPS,
    save_total_limit=1,
    logging_dir="./logs",
    logging_strategy=IntervalStrategy.STEPS,
    logging_steps=250,
    eval_steps=250,
    load_best_model_at_end=True,
    lr_scheduler_type="cosine_with_restarts",
    warmup_steps=250,
  )
    return training_args

In [ ]:
def evaluate_func(trainer):
  trainer.evaluate()
  predictions = trainer.predict(valid_dataset)
  predicted_logits = predictions.predictions
  print(f"Type of predicted_logits: {type(predicted_logits)}")
  if isinstance(predicted_logits, tuple):
        predicted_logits = predicted_logits[0]
  if not isinstance(predicted_logits, np.ndarray):
        predicted_logits = np.array(predicted_logits)
  if len(predicted_logits.shape) == 3:
        predicted_logits = predicted_logits[:, 0, :]
  predicted_probabilities = expit(predicted_logits)
  threshold = 0.5
  predicted_classes = (predicted_probabilities >= threshold).astype(int)
  precision, recall, f1, _ = precision_recall_fscore_support(y_valid, predicted_classes, average=None)
  accuracy = accuracy_score(y_valid, predicted_classes)
  return precision, recall, f1,accuracy

In [ ]:
MAX_LENGTH = 256
metrics_df = pd.DataFrame(columns=["Models_name", "Precision", "Recall", "F1_score", "Accuracy"])
confusion_matrices = []
n_splits = 5
mskf = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1)

In [ ]:
models_list=models_list_1
models_name=models_name_1

# models_list=models_list_2
# models_name=models_name_2
print(models_list)
for x in range(len(models_list)):
  torch.cuda.empty_cache()

  i=models_list[x]
  print(i)
  tokenizer = AutoTokenizer.from_pretrained(i)
  model = AutoModelForSequenceClassification.from_pretrained(i, num_labels=5, ignore_mismatched_sizes=True)
  device="cuda" if torch.cuda.is_available() else 'cpu'
  model = model.to(device)
  for param in model.parameters():
      param.data = param.data.contiguous()

  training_args = train_arg()
  train_dataset,valid_dataset=build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH)
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=valid_dataset,
  )
  trainer.train()
  precision, recall, f1,accuracy=evaluate_func(trainer)
  new_data = pd.DataFrame({
    "Models_name": [models_name[x]],
    "Precision": [precision],
    "Recall": [recall],
    "F1_score": [f1],
    "Accuracy": [accuracy]
  })

  metrics_df = pd.concat([metrics_df, new_data], ignore_index=True)

  model_confusion_matrices = []
  predicted_classes = trainer.predict(valid_dataset).predictions



  if isinstance(predicted_classes, tuple) :
    predicted_classes = predicted_classes[0]
  predicted_classes = np.array(predicted_classes)
  # print(f"Type after conversion: {type(predicted_classes)}")

  if len(predicted_classes.shape) > 2:
    predicted_classes = predicted_classes.reshape(predicted_classes.shape[0], -1)
    # print(f"Shape after reshaping: {predicted_classes.shape}")




  predicted_classes = (expit(predicted_classes) >= 0.5).astype(int)

  y_valid_np = np.array(y_valid)

  for j in range(len(label_columns)):
      print(f"Computing confusion matrix for label: {label_columns[j]}")
      cm = confusion_matrix(y_valid_np[:, j], predicted_classes[:, j])

      # cm = confusion_matrix(y_valid[:, j], predicted_classes[:, j])
      model_confusion_matrices.append(cm)
  confusion_matrices.append(model_confusion_matrices)


  print(f"{models_name[x]} evaluates:")
  for k, label in enumerate(label_columns):
    print(f"Class '{label}': Precision: {precision[k]:.4f}, Recall: {recall[k]:.4f}, F1: {f1[k]:.4f}")
  print(f"Overall accuracy: {accuracy:.4f}")
  print("-------------------------------------------------------------------------\n")

['funnel-transformer/medium', 'google/electra-base-discriminator', 'facebook/bart-base']
funnel-transformer/medium


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

...

Step,Training Loss,Validation Loss
250,0.504400,0.438956


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:384: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}
  warnings.warn(


Type of predicted_logits: <class 'tuple'>


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Computing confusion matrix for label: Anger
Computing confusion matrix for label: Fear
Computing confusion matrix for label: Joy
Computing confusion matrix for label: Sadness
Computing confusion matrix for label: Surprise
bart-base evaluates:
Class 'Anger': Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Class 'Fear': Precision: 0.7405, Recall: 0.9009, F1: 0.8128
Class 'Joy': Precision: 0.6744, Recall: 0.4296, F1: 0.5249
Class 'Sadness': Precision: 0.7094, Recall: 0.4716, F1: 0.5666
Class 'Surprise': Precision: 0.6620, Recall: 0.5629, F1: 0.6084
Overall accuracy: 0.3520
-------------------------------------------------------------------------

